# SQLAlchemy tutorial

Para ejecutar este notebook debe de instalar...

    pip install -r requirements.txt
    pip install ipykernel
    pip install python-dotenv

In [ ]:
# SQLAlchemy version
import sqlalchemy

sqlalchemy.__version__

In [ ]:
# Cargar variables de entorno
import os
from dotenv import load_dotenv

load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
print(DB_HOST, DB_NAME, DB_USER)

In [ ]:
# Engine con MySQL
from sqlalchemy import create_engine

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}", echo=False, future=True)
print(type(engine))

In [ ]:
# DROP TABLE
from sqlalchemy import text

with engine.connect() as conn:
    resultado = conn.execute(text("DROP TABLE IF EXISTS estados"))
    resultado = conn.execute(text("DROP TABLE IF EXISTS usuarios"))
    resultado = conn.execute(text("DROP TABLE IF EXISTS roles"))

In [ ]:
# CREATE TABLE
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class Rol(Base):
    __tablename__ = "roles"
    id = Column(Integer, primary_key=True)
    nombre = Column(String(256), unique=True, nullable=False)
    permiso = Column(Integer, nullable=False)
    usuarios = relationship("Usuario", back_populates="rol")

class Usuario(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True)
    rol_id = Column(Integer, ForeignKey("roles.id"), index=True, nullable=False)
    rol = relationship("Rol", back_populates="usuarios")
    email = Column(String(256), unique=True, index=True)
    nombres = Column(String(256), nullable=False)
    apellido_paterno = Column(String(256), nullable=False)
    apellido_materno = Column(String(256))
    contrasena = Column(String(256), nullable=False)

class Estado(Base):
    __tablename__ = "estados"
    id = Column(Integer, primary_key=True)
    nombre = Column(String(256), unique=True, nullable=False)

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [ ]:
# Sesion
from sqlalchemy.orm import sessionmaker

session_local = sessionmaker(autocommit=False, autoflush=False, bind=engine)
print(type(session_local))

In [ ]:
# INSERT roles
with session_local() as db:
    db.add(Rol(nombre="ADMINISTRADOR", permiso=1))
    db.add(Rol(nombre="OBSERVADOR", permiso=1))
    db.commit()

In [ ]:
# Query roles
with session_local() as db:
    consulta = db.query(Rol).all()
    for rol in consulta:
        print(rol.nombre)

In [ ]:
# Cifrar contraseñas
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["pbkdf2_sha256", "des_crypt"], deprecated="auto")

secreto_1 = pwd_context.hash("AlgoSecreto")
secreto_2 = pwd_context.hash("AlgoSecreto")
print(secreto_1)
print(secreto_2)

if secreto_1 == secreto_2:
    print("ESTO NO SE VA A VER.")

if pwd_context.verify("AlgoSecreto", secreto_1):
    print("Las contrasenas es la correcta.")

if pwd_context.verify("AlgoSecreto", secreto_2):
    print("Las contrasenas es la correcta.")

In [ ]:
# INSERT usuarios
with session_local() as db:
    rol = db.query(Rol).get(1)
    db.add(Usuario(rol=rol, nombres="Guillermo", apellido_paterno="Valdes", contrasena=pwd_context.hash("AlgoSecreto")))
    db.commit()

In [ ]:
# Query usuarios
with session_local() as db:
    for usuario in db.query(Usuario).all():
        print(usuario.nombres, usuario.apellido_paterno)

In [ ]:
# INSERT estados
import csv

with session_local() as db:
    with open("../seed/estados.csv", "r", encoding="utf8") as puntero:
        for renglon in csv.DictReader(puntero):
            db.add(Estado(nombre=renglon["nombre"]))
    db.commit()

In [ ]:
# Query estados
with session_local() as db:
    for estado in db.query(Estado).all():
        print(estado.nombre)